In [2]:
import pandas as pd
import numpy as np
import torch
torch.cuda.is_available()

True

In [3]:
movie_complete = pd.read_csv('../TMDB_movie_dataset_v11.csv')


In [4]:
movie_complete.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [5]:
movie_complete.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [12]:
# title, overview,
# adult, genres, keywords
np.random.seed(42)
movie = movie_complete[['adult','genres','keywords','title','overview']].dropna()
movie = movie.sample(10000)

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

In [14]:
def make_input_prompt(row):
    return (f"adult: {row['adult']}\n"
            f"genres: {row['genres']}\n"
            f"keywords: {row['keywords']}\n"
            "Generate title and overview:")

def make_target_text(row):
    return (f"title: {row['title']}\n"
            f"overview: {row['overview']}")

movie['input_text'] = movie.apply(make_input_prompt, axis=1)
movie['target_text'] = movie.apply(make_target_text, axis=1)
movie.head()

,adult,genres,keywords,title,overview,input_text,target_text
414707,False,Music,"ballet, filmed theater",Royal Opera House 2023/24: Macmillan,The Royal Ballet celebrates the breadth of Pri...,"adult: False\ngenres: Music\nkeywords: ballet,...",title: Royal Opera House 2023/24: Macmillan\no...
378097,False,"Comedy, Adventure, Animation, Fantasy","magic, wish",In Your Dreams,Stevie and her brother Elliot magically travel...,"adult: False\ngenres: Comedy, Adventure, Anima...",title: In Your Dreams\noverview: Stevie and he...
16716,False,"Horror, Thriller","isolation, gas station, delusion, hallucinatio...",Open 24 Hours,After setting her serial killer boyfriend on f...,"adult: False\ngenres: Horror, Thriller\nkeywor...",title: Open 24 Hours\noverview: After setting ...
757361,False,Music,"rosalía, lost in translation, motomami",Candy,"The colorful video, directed by Stillz, follow...",adult: False\ngenres: Music\nkeywords: rosalía...,"title: Candy\noverview: The colorful video, di..."
90943,False,Animation,"olympic games, cartoon, sigmund freud, surreal...",Sigmund,A young boy is transformed as he watches the O...,adult: False\ngenres: Animation\nkeywords: oly...,title: Sigmund\noverview: A young boy is trans...


In [15]:
# Simple random split
train_df = movie.sample(frac=0.8, random_state=42)
val_df = movie.drop(train_df.index)

In [16]:
train_dataset = Dataset.from_pandas(train_df[['input_text','target_text']])
val_dataset = Dataset.from_pandas(val_df[['input_text','target_text']])

tv_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [ ]:
model_name = "t5-small"  # or "google/flan-t5-base", etc.
tokenizer = T5Tokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    # We feed the model the input_text, and want to predict target_text
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=256)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_datasets = tv_dict.map(tokenize_function, batched=True)

Map: 100%|██████████| 2000/2000 [00:00<00:00, 3082.75 examples/s]


In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="t5_movies",
    eval_strategy="steps",
    eval_steps=500,  # adjust
    per_device_eval_batch_size=15,
    learning_rate=3e-5,
    per_device_train_batch_size=15,  # 16 max?
    per_gpu_train_batch_size= 15,
    num_train_epochs=2,  # ideally 3
    weight_decay=0.01,
    logging_steps=500,
    save_steps=500,
    fp16=True, 
    save_total_limit=2,
    push_to_hub=False, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [35]:
trainer.train()
trainer.save_model("t5_movies")
tokenizer.save_pretrained("t5_movies")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [65]:
tokenizer = T5Tokenizer.from_pretrained("t5_movies")
model = T5ForConditionalGeneration.from_pretrained("t5_movies")

def generate_tv_show_description(adult, genres, keywords):
    # Create a prompt
    prompt = (f"adult: {adult}\n"
            f"genres: {genres}\n"
            f"keywords: {keywords}\n"
            "Generate title and overview:")

    # Tokenize
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # Generate
    output_ids = model.generate(
        input_ids,
        max_length=256,
        do_sample=True,
        top_p=0.9,       # nucleus/top-p sampling
        top_k=50,
        temperature=1.0,
        repetition_penalty=1.2
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

# Example usage
test_adult = False
test_genres = "Action, Thriller, Adventure"
test_keywords = "jungle, treasure, expedition"

result = generate_tv_show_description(test_adult, test_genres, test_keywords)
print(result)

title: A tv overview: An explorer who is going to have a jungle break through his homeland would encounter a cynical adventure – the perfect backdrop for an expedition.
